# **1. Prompt + LLM**

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os


prompt = ChatPromptTemplate.from_template(
    "Hãy trình bày tổng qua về đát nước {foo}. Lưu ý: Bạn chỉ được phép sử dụng tiếng việt để trả lời"
)
model = ChatGroq(
    model="llama3-70b-8192",
    api_key= os.getenv("GROQ_API_KEY")
)

chain = prompt | model

response = chain.invoke({"foo": "Nga"})
print(response.content)

Tôi vui lòng trình bày tổng quan về nước Nga như sau:

**Vị trí địa lý**: Nga là quốc gia lớn nhất thế giới về diện tích, nằm tại phần bắc của lục địa Á-Âu. Nó giáp với 14 quốc gia, bao gồm Na Uy, Phần Lan, Estonia, Latvia, Litva, Ba Lan, Belarus, Ukraine, Gruzia, Azerbaijan, Kazakhstan, Trung Quốc, Mông Cổ và Triều Tiên.

**Địa hình**: Nga có địa hình đa dạng, từ các đồng bằng rộng lớn đến các dãy núi cao, bao gồm dãy Ural, dãy Caucasus và dãy Altai. Phần lớn diện tích nước Nga là đồng bằng, với các sông lớn như Volga, Ob, Yenisei và Amur.

**Khí hậu**: Nga có khí hậu ôn đới lục địa, với mùa đông lạnh và mùa hè ấm áp. Nhiệt độ trung bình trong năm thay đổi từ -50°C đến 20°C, tùy thuộc vào vùng.

**Dân số và ngôn ngữ**: Nga có dân số khoảng 146 triệu người, chủ yếu là người Nga. Ngoài ra, còn có hơn 180 dân tộc thiểu số, bao gồm người Tatar, người Ukraina, người Belarus và người Kazakh. Tiếng Nga là ngôn ngữ chính thức, nhưng còn có nhiều ngôn ngữ thiểu số khác được nói.

**Thủ đô và c

# **2. Prompt + LLM + OutputParser**

In [11]:
from langchain.schema.output_parser import StrOutputParser
chain = prompt | model | StrOutputParser()

response = chain.invoke({"foo": "Nga"})
print(response)


Tôi sẵn sàng!

Nước Nga, tên đầy đủ là Liên bang Nga, là một quốc gia nằm tại Bắc Âu và Đông Âu. Đây là quốc gia lớn nhất thế giới về diện tích, với tổng diện tích khoảng 17,1 triệu km², chiếm khoảng 1/8 diện tích đất liền của Trái Đất.

Vị trí địa lý: Nước Nga giáp với nhiều quốc gia, bao gồm Na Uy, Phần Lan, Estonia, Latvia, Litva, Belarus, Ukraine, Gruzia, Azerbaijan, Kazakhstan, Trung Quốc, Mông Cổ và Triều Tiên. Nước Nga cũng giáp với các biển Baltic, biển Trắng, biển Barents, biển Kara, biển Laptev, biển Đông Siberia và biển Okhotsk.

Khí hậu: Nước Nga có khí hậu đa dạng, từ khí hậu ôn đới ở phía tây đến khí hậu cực寒 ở phía bắc và khí hậu lục địa ở phía đông. Khí hậu tại Nga được chia thành bốn mùa: xuân, hạ, thu và đông.

Thành phố: Thủ đô của Nga là Moscow, thành phố lớn nhất và là trung tâm kinh tế, chính trị, văn hóa của nước này. Thành phố Saint Petersburg là thành phố lớn thứ hai của Nga và là thành phố lịch sử, văn hóa quan trọng.

Kinh tế: Nước Nga là một trong những nước

# **3. RAG (Retrieval-augmented Generation)**

In [5]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import FastEmbedEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.vectorstores import FAISS

embedding_model = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# Create a vector store and retriever
vectorstore = FAISS.from_texts(
    ["Nước Nga, tên đầy đủ là Liên bang Nga, là một quốc gia nằm tại Bắc Âu và Đông Âu.",
     " Đây là quốc gia lớn nhất thế giới về diện tích, với tổng diện tích khoảng 17,1 triệu km², chiếm khoảng 1/8 diện tích đất liền của Trái Đất.",
     "Thủ đô của Nga là Moscow, thành phố lớn nhất và là trung tâm kinh tế, chính trị, văn hóa của nước này."], embedding=embedding_model
)
retriever = vectorstore.as_retriever()

# Define templates for prompts
template = """Trả lời câu hỏi dự vào thông tin context bên dưới:
{context}
Lưu ý: bạn chỉ được trả lời bằng tiếng việt.
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatGroq(
    model="llama3-70b-8192",
    api_key= os.getenv("GROQ_API_KEY")
)

# Create a retrieval-augmented generation chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

result = chain.invoke("tên đầy đủ của nước Nga là gì ?")

print(result)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Tên đầy đủ của nước Nga là Liên bang Nga.


# **4. Multiple Chains**

In [18]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt1 = ChatPromptTemplate.from_template("{person} đến từ thành phó nào?")
prompt2 = ChatPromptTemplate.from_template(
    "{city} ở nước nào? Trả lời câu hỏi bằng {language}"
)

model = ChatGroq(
    model="llama3-70b-8192",
    api_key= os.getenv("GROQ_API_KEY")
)

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

result = chain2.invoke({"person": "Hồ Chí Minh", "language": "Việt Nam"})
print(result)

Hồ Chí Minh đến từ Việt Nam.


## **5. Branching and Merging**

In [23]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

model = ChatGroq(
    model="llama3-70b-8192",
    api_key= os.getenv("GROQ_API_KEY")
)

planner = (
    ChatPromptTemplate.from_template("Tạo lập luận về: {input}. Lưu ý: bạn chỉ được sử dụng tiếng việt ")
    | model
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "Liệt kê những ưu điểm hoặc khía cạnh tích cực của {base_response}. Lưu ý: bạn chỉ được sử dụng tiếng việt"
    )
    | model
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "Liệt kê những nhược điểm hoặc khía cạnh tiêu cực của {base_response}. Lưu ý: bạn chỉ được sử dụng tiếng việt"
    )
    | model
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Tạo phản hồi cuối cùng dựa trên lời phê bình"),
        ]
    )
    | model
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

result = chain.invoke({"input": "mô hình ngôn ngữ lớn"})
print(result)

Here is a rewritten version of the text that combines the pros and cons into a single, cohesive piece:

**Mô hình ngôn ngữ lớn: Xu hướng và tiềm năng trong tương lai**

Các mô hình ngôn ngữ lớn đã trở thành một xu hướng và tiềm năng to lớn trong tương lai. Chúng có thể được sử dụng để phát triển các hệ thống và ứng dụng hiệu quả, giúp các doanh nghiệp và tổ chức có thể giao tiếp và hoạt động một cách hiệu quả hơn.

**Ưu điểm của mô hình ngôn ngữ lớn**

Các mô hình ngôn ngữ lớn có nhiều lợi ích và tiềm năng to lớn. Chúng có thể được sử dụng để phát triển các hệ thống giáo dục trực tuyến, giúp các học sinh và sinh viên có thể học tập một cách hiệu quả và tiện lợi hơn. Chúng cũng có thể được sử dụng để phát triển các hệ thống chăm sóc sức khỏe, giúp các bác sĩ và nhân viên y tế có thể chẩn đoán và điều trị bệnh một cách hiệu quả hơn. Ngoài ra, chúng có thể được sử dụng để phát triển các hệ thống an ninh, giúp các cơ quan chức năng có thể phát hiện và phòng chống tội phạm một cách hiệu quả

# **6. Adding Memory to a Chain**

In [7]:
from langchain.schema import SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
import os

# Create a ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="Bạn là một chatbot hữu ích đang trò chuyện với con người. Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện"
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will be injected
    ]
)

llm = ChatGroq(
    model="llama3-70b-8192",
    api_key= os.getenv("GROQ_API_KEY")
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = LLMChain(
    llm=llm, 
    prompt=prompt,
    verbose=True,
    memory=memory
)

response = chain.predict(human_input="Việt Nam có bao nhiêu dân tộc ?")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: Bạn là một chatbot hữu ích đang trò chuyện với con người. Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện
Human: Việt Nam có bao nhiêu dân tộc ?

> Finished chain.
Xin chào! Việt Nam là một đất nước đa dân tộc, với tổng cộng 54 dân tộc anh em. Trong đó, dân tộc Kinh chiếm đa số, khoảng 85,3% dân số, và 53 dân tộc thiểu số khác. Mỗi dân tộc đều có ngôn ngữ, văn hóa, truyền thống và phong tục riêng, góp phần vào sự đa dạng và giàu có của văn hóa Việt Nam.


In [36]:
response = chain.predict(human_input="Tôi vừa hỏi bạn điều gì ?")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: Bạn là một chatbot hữu ích đang trò chuyện với con người. Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện
Human: Việt Nam có bao nhiêu dân tộc ?
AI: Xin chào! Việt Nam chúng ta có 54 dân tộc anh em, mỗi dân tộc có nét văn hóa, truyền thống và ngôn ngữ riêng. Đó là một sự đa dạng và phong phú về văn hóa của Việt Nam.
Human: Tôi vừa hỏi bạn điều gì ?

> Finished chain.
Bạn vừa hỏi tôi "Việt Nam có bao nhiêu dân tộc?"


# **7. Routing by Semantic Similarity**

In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.utils.math import cosine_similarity

physics_template = """Bạn là một giáo sư vật lý rất thông minh. \ 
Bạn rất giỏi trả lời các câu hỏi về vật lý một cách ngắn gọn và dễ hiểu. \ 
Khi bạn không biết câu trả lời cho một câu hỏi, bạn thừa nhận rằng bạn không biết
Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện
Here is a question:
{query}"""

math_template = """Bạn là một nhà toán học rất giỏi. Bạn rất giỏi trả lời các câu hỏi toán học. \ 
Bạn thật giỏi vì bạn có thể chia nhỏ các vấn đề khó khăn thành các phần cấu thành của chúng, \ 
trả lời các phần cấu thành và sau đó ghép chúng lại với nhau để trả lời câu hỏi rộng hơn.
Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện
Here is a question:
{query}"""

embedding_model = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")

prompt_templates = [physics_template, math_template]
prompt_embeddings = embedding_model.embed_documents(prompt_templates)

def prompt_router(input):
    query_embedding = embedding_model.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PROGRAM")
    return PromptTemplate.from_template(most_similar)

chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)

print(chain.invoke({"query": "Lỗ đen là gì?"}))
print(chain.invoke({"query": "Ngôn ngữ lập trình Java để làm gì?"}))

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Using PROGRAM
Lỗ đen là một vùng trong không gian, nơi trọng lực quá mạnh đến mức không có gì, kể cả ánh sáng, có thể thoát ra được. Nó được hình thành khi một ngôi sao lớn bị sụp đổ và mật độ của nó tăng lên đến mức cực kỳ cao. Lỗ đen có một ranh giới gọi là chân trời sự kiện, bất cứ vật gì vượt qua ranh giới này đều sẽ bị hút vào lỗ đen và không thể thoát ra được. Lỗ đen là một trong những hiện tượng kỳ lạ và phức tạp nhất trong vũ trụ.
Using PROGRAM
Tôi xin lỗi, nhưng tôi là giáo sư vật lý, không phải là giáo sư tin học. Tôi không có chuyên môn về ngôn ngữ lập trình Java. Tôi không biết câu trả lời cho câu hỏi này. Java là một ngôn ngữ lập trình thuộc về lĩnh vực công nghệ thông tin, không thuộc về vật lý. Tôi chỉ có thể trả lời các câu hỏi về vật lý, không phải về lập trình.
